## Import thư viện PyTorch

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Thiết lập device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Đặt các siêu tham số (Hyper-parameters)

In [ ]:
# Độ dài chuỗi
sequence_length = 28

# Kích thước đầu vào
input_size = 28

# Kích thước ẩn
hidden_size = 128

# Số lượng tầng
num_layers = 2

# Số lượng lớp
num_classes = 10

# Kích thước mỗi batch
batch_size = 100

# Số lượng epochs
num_epochs = 2

# Tốc độ học
learning_rate = 0.003

## Load bộ dữ liệu MNTST

In [ ]:
# Dữ liệu tập huấn luyện
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

# Dữ liệu tập kiểm tra
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

## DataLoader

In [ ]:
# DataLoader cho tập huấn luyện
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

# DataLoader cho tập kiểm tra
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Mô hình Bidirectional recurrent neural network (many-to-one)

In [ ]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Khởi tạo các trạng thái
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Lan truyền tiến (Forward) LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Giải mã (Decode) trạng thái ẩn của time step cuối cùng
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)
model

## Hàm mất mát (loss function) và bộ tối ưu hóa (optimizer)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Huấn luyện mô hình

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Lan truyền tiến (Forward pass) và tính toán mất mát
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Lan truyền ngược (Backward pass) và tối ưu trọng số
        ## Đặt zero_grad để không tính toán đạo hàm
        optimizer.zero_grad()
        ## Gọi backward()
        loss.backward()
        ## Tối ưu
        optimizer.step()
        # In ra thông tin huấn luyện mỗi 100 iter
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

## Kiểm tra mô hình

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

## Lưu mô hình (model checkpoint)

In [ ]:
torch.save(model.state_dict(), 'model.ckpt')